## Memory
- Conversation Chain
- Conversation Buffer Memory

In [1]:
import os 
import logging 

from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())
api_key = os.environ.get('OPENAI_API_KEY')
if api_key is None:
    raise ValueError("The OPENAI_API_KEY environment variable is not set.")

In [2]:
import os 
from langchain_openai import ChatOpenAI
chat = ChatOpenAI(
    openai_api_base=os.environ["CHATGPT_API_ENDPOINT"],
    openai_api_key=os.environ["OPENAI_API_KEY"])

/Users/babyching/Documents/personal/LLM/從LangChain接入ChatGPT到製作股票分析AI團隊/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [3]:
from langchain.chains import ConversationChain
conversation = ConversationChain(
    llm=chat
)
print(conversation.prompt.template)

The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
{history}
Human: {input}
AI:


In [4]:
from langchain.memory import ConversationBufferMemory
conversation_buf= ConversationChain(
    llm=chat,
    memory=ConversationBufferMemory()
)

In [5]:
response = conversation_buf.invoke({"input": "你知道什麼是IG嗎?"})
print(response)

{'input': '你知道什麼是IG嗎?', 'history': '', 'response': '當然知道！IG是指Instagram的縮寫，是一個社交媒體平台，用戶可以分享照片和視頻，與朋友和關注者互動。IG在全球非常流行，許多名人和品牌都在上面活躍。您有IG帳號嗎？'}


In [6]:
response = conversation_buf.invoke({"input": "你知道如何在Goolge Chrome上面添加Tab Group嗎?"})
print(response)

{'input': '你知道如何在Goolge Chrome上面添加Tab Group嗎？', 'history': 'Human: 你知道什麼是IG嗎?\nAI: 當然知道！IG是指Instagram的縮寫，是一個社交媒體平台，用戶可以分享照片和視頻，與朋友和關注者互動。IG在全球非常流行，許多名人和品牌都在上面活躍。您有IG帳號嗎？', 'response': '當然知道！在Google Chrome上添加Tab Group非常容易。您只需右鍵點擊標籤欄上的標籤，然後選擇"將標籤添加到新群組"。接著您可以為新的Tab Group命名，並將相關的標籤拖曳到群組中。這樣可以更好地組織您的標籤，方便您管理和瀏覽網頁。您可以試試看！'}


In [7]:
print(conversation_buf.memory.buffer)

Human: 你知道什麼是IG嗎?
AI: 當然知道！IG是指Instagram的縮寫，是一個社交媒體平台，用戶可以分享照片和視頻，與朋友和關注者互動。IG在全球非常流行，許多名人和品牌都在上面活躍。您有IG帳號嗎？
Human: 你知道如何在Goolge Chrome上面添加Tab Group嗎？
AI: 當然知道！在Google Chrome上添加Tab Group非常容易。您只需右鍵點擊標籤欄上的標籤，然後選擇"將標籤添加到新群組"。接著您可以為新的Tab Group命名，並將相關的標籤拖曳到群組中。這樣可以更好地組織您的標籤，方便您管理和瀏覽網頁。您可以試試看！


### Conversation summary
將對話做總結不是把所有內容都記錄下來
- 在token上可以減少

In [10]:
from langchain.memory import ConversationSummaryMemory
conversation_sum = ConversationChain(
    llm=chat,
    memory = ConversationSummaryMemory(llm=chat)
)

In [12]:
print(conversation_sum.memory.prompt.template)

Progressively summarize the lines of conversation provided, adding onto the previous summary returning a new summary.

EXAMPLE
Current summary:
The human asks what the AI thinks of artificial intelligence. The AI thinks artificial intelligence is a force for good.

New lines of conversation:
Human: Why do you think artificial intelligence is a force for good?
AI: Because artificial intelligence will help humans reach their full potential.

New summary:
The human asks what the AI thinks of artificial intelligence. The AI thinks artificial intelligence is a force for good because it will help humans reach their full potential.
END OF EXAMPLE

Current summary:
{summary}

New lines of conversation:
{new_lines}

New summary:


In [13]:
conversation_sum.invoke({"input":"你知道什麼是IG嗎?"})

{'input': {'input:你知道什麼是IG嗎?'},
 'history': '',
 'response': '知道呀!IG指的是Instagram，一個非常流行的社交媒體應用程式，用戶可以在上面分享照片和影片，和朋友互動，關注自己感興趣的內容。你有IG帳號嗎？'}

In [14]:
conversation_sum.invoke({"input":"你知道如何在Goolge Chrome上面添加Tab Group嗎?"})

{'input': '你知道如何在Goolge Chrome上面添加Tab Group嗎?',
 'history': "The human asks if the AI knows what IG is. The AI responds that IG refers to Instagram, a popular social media app where users can share photos and videos, interact with friends, follow content they're interested in. The AI asks if the human has an IG account.",
 'response': '當然知道！在Google Chrome上添加Tab Group非常簡單。您只需右鍵單擊要分組的選項卡，然後選擇“新增到新群組”。接下來，您可以為該組選項卡指定一個名稱和顏色，以便更容易識別和組織它們。您可以在同一群組中添加多個選項卡，並且可以輕鬆地將它們拖放到不同的位置。希望這可以幫到您！'}

In [15]:
print(conversation_sum.memory.buffer)

The human asks if the AI knows what IG is. The AI responds that IG refers to Instagram and asks if the human has an IG account. The human then asks the AI if it knows how to add Tab Groups on Google Chrome. The AI explains that adding Tab Groups on Google Chrome is simple and provides step-by-step instructions on how to do so.


In [16]:
from langchain.memory import ConversationBufferWindowMemory

conversation_buff_win = ConversationChain(
    llm = chat,
    memory = ConversationBufferWindowMemory(k=1)
)

In [17]:
conversation_buff_win.invoke({"input":"你知道如何在Goolge Chrome上面添加Tab Group嗎?"})

{'input': '你知道如何在Goolge Chrome上面添加Tab Group嗎?',
 'history': '',
 'response': '當然！在Google Chrome中，您可以輕鬆地添加Tab Group。只需右鍵點擊任何標籤，然後選擇"新增標籤組"，您可以為新組命名並將其他標籤拖放到該組中。這樣您就可以更有效地組織和管理您的瀏覽器標籤了！您可以試試看！'}

In [18]:
conversation_buff_win.invoke({"input":"如何在Youtube實現畫中畫功能嗎？"})

{'input': '如何在Youtube實現畫中畫功能嗎？',
 'history': 'Human: 你知道如何在Goolge Chrome上面添加Tab Group嗎?\nAI: 當然！在Google Chrome中，您可以輕鬆地添加Tab Group。只需右鍵點擊任何標籤，然後選擇"新增標籤組"，您可以為新組命名並將其他標籤拖放到該組中。這樣您就可以更有效地組織和管理您的瀏覽器標籤了！您可以試試看！',
 'response': '畫中畫功能在YouTube上非常方便使用。當您觀看任何視頻時，只需將滑鼠右鍵點擊視頻播放器，然後選擇"畫中畫"選項。這樣視頻就會在一個小窗口中繼續播放，您可以在其他標籤或應用程序中繼續瀏覽，同時不會錯過任何內容。該功能非常方便，您可以隨時試試看！'}

In [19]:
conversation_buff_win.invoke({"input":"如何做Pizza?"})

{'input': '如何做Pizza?',
 'history': 'Human: 如何在Youtube實現畫中畫功能嗎？\nAI: 畫中畫功能在YouTube上非常方便使用。當您觀看任何視頻時，只需將滑鼠右鍵點擊視頻播放器，然後選擇"畫中畫"選項。這樣視頻就會在一個小窗口中繼續播放，您可以在其他標籤或應用程序中繼續瀏覽，同時不會錯過任何內容。該功能非常方便，您可以隨時試試看！',
 'response': '製作比薩餅非常簡單！首先，您需要準備比薩餅皮。您可以在超市購買現成的比薩餅皮，或者自己動手做。接著，您可以在比薩餅皮上塗抹番茄醬，撒上您喜歡的配料，比如蔬菜、香腸、起司等等。最後，將比薩放入烤箱中烤至表面金黃酥脆即可。這樣一個美味的比薩就完成了！您也可以根據自己的口味和喜好進行變化，製作出不同口味的比薩。祝您做比薩成功！'}

In [20]:
print(conversation_buff_win.memory.buffer)

Human: 如何做Pizza?
AI: 製作比薩餅非常簡單！首先，您需要準備比薩餅皮。您可以在超市購買現成的比薩餅皮，或者自己動手做。接著，您可以在比薩餅皮上塗抹番茄醬，撒上您喜歡的配料，比如蔬菜、香腸、起司等等。最後，將比薩放入烤箱中烤至表面金黃酥脆即可。這樣一個美味的比薩就完成了！您也可以根據自己的口味和喜好進行變化，製作出不同口味的比薩。祝您做比薩成功！
